# Analisis Sentimen Twitter - Naive Bayes

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [ ]:

# Load Dataset
df = pd.read_csv('/content/Tweets.csv')  # Ubah path sesuai lokasi dataset
df = df[['text', 'airline_sentiment']]
df = df[df['airline_sentiment'].isin(['positive', 'negative', 'neutral'])]
df = df.dropna()

print(df['airline_sentiment'].value_counts())
df.head()


In [ ]:

# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'http\S+|@\S+|#\S+|[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(preprocess)


In [ ]:

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_text']).toarray()
y = df['airline_sentiment']


In [ ]:

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeling
model = MultinomialNB()
model.fit(X_train, y_train)


In [ ]:

# Evaluation
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
plt.figure(figsize=(6,5))
cm = confusion_matrix(y_test, y_pred, labels=['positive','neutral','negative'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['positive','neutral','negative'],
            yticklabels=['positive','neutral','negative'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:

# Sample Predictions
samples = df.sample(5)
sample_features = tfidf.transform(samples['clean_text'])
preds = model.predict(sample_features)

for i in range(len(samples)):
    print(f"\nTweet: {samples['text'].values[i]}")
    print(f"Predicted: {preds[i]}")
    print(f"Actual: {samples['airline_sentiment'].values[i]}")
